In [1]:
import json
import numpy as np
import random
import nltk
# import svgling
import os
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download([
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
    "maxent_ne_chunker",
    "words",
])

[nltk_data] Downloading package names to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yingjie_niu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to


True

In [2]:
testset0 = json.load(open('../../data/NewsQA/testsets/newsQA_test_0.json', 'r', encoding='utf-8'))
testset1 = json.load(open('../../data/NewsQA/testsets/newsQA_test_1.json', 'r', encoding='utf-8'))
testset2 = json.load(open('../../data/NewsQA/testsets/newsQA_test_2.json', 'r', encoding='utf-8'))
testset3 = json.load(open('../../data/NewsQA/testsets/newsQA_test_3.json', 'r', encoding='utf-8'))
testset4 = json.load(open('../../data/NewsQA/testsets/newsQA_test_4.json', 'r', encoding='utf-8'))


In [2]:
# qa_dataset = testset0+testset1+testset2+testset3+testset4
qa_dataset = json.load(open('../../data/NewsQA/processed_newsQA.json', 'r', encoding='utf-8'))
qa_dataset[0]

{'context': 'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main

### Question Type Template

In [3]:
print('Number of qa pairs: ', len(qa_dataset))
question_list = []
for qa in qa_dataset: 
    question_list.append(qa['question'])

Number of qa pairs:  31423


In [4]:
what = {'count':0, 'list':[]}
why = {'count':0, 'list':[]}
where = {'count':0, 'list':[]}
when = {'count':0, 'list':[]}
how = {'count':0, 'list':[]}
how_much = {'count':0, 'list':[]}
how_many = {'count':0, 'list':[]}
which = {'count':0, 'list':[]}
who = {'count':0, 'list':[]}
other = {'count':0, 'list':[]}

for query in question_list:
    query = query.capitalize()
    if 'What' in query:
        what['count'] += 1
        what['list'].append(query)
    elif 'Why' in query:
        why['count'] += 1
        why['list'].append(query)
    elif 'Where' in query:
        where['count'] += 1
        where['list'].append(query)
    elif 'When' in query:
        when['count'] += 1
        when['list'].append(query)
    elif 'How much' in query:
        how_much['count'] += 1
        how_much['list'].append(query)
    elif 'How many' in query:
        how_many['count'] += 1
        how_many['list'].append(query)
    elif 'How' in query:
        how['count'] += 1
        how['list'].append(query)
    elif 'Which' in query:
        which['count'] += 1
        which['list'].append(query)
    elif 'Who' in query:
        who['count'] += 1
        who['list'].append(query)
    else:
        other['count'] += 1
        other['list'].append(query)
print('Why: ',why['count'], 'What: ',what['count'], 'Where',where['count'], 'When',when['count'], 'How',how['count'], 'How many', how_many['count'], 'How much', how_much['count'], 'Which',which['count'],'Who',who['count'], 'Other', other['count'])
print(why['count']+what['count']+where['count']+when['count']+how['count']+how_much['count']+how_many['count']+which['count']+who['count']+other['count'])
print(len(question_list))

Why:  28 What:  14528 Where 2523 When 1369 How 279 How many 1741 How much 303 Which 770 Who 7201 Other 2681
31423
31423


### Add Qtype template to dataset

In [5]:
def count_words(dataset):
    words = {}
    for i in range(len(dataset)):
        for word in dataset[i].split():
            if word not in words:
                words[word]=1
            else:
                words[word]+=1
    return words

def sort_wordCounter(words_dict):
    # input: a dict of words, key:word, value:times of word shown in the context
    # output: a list sorted by the value of dict
    a = dict(sorted(words_dict.items(), key=lambda x:x[1], reverse=True))
    return a


In [6]:
what = {'count':0, 'list':[]}
why = {'count':0, 'list':[]}
where = {'count':0, 'list':[]}
when = {'count':0, 'list':[]}
how = {'count':0, 'list':[]}
how_much = {'count':0, 'list':[]}
how_many = {'count':0, 'list':[]}
which = {'count':0, 'list':[]}
who = {'count':0, 'list':[]}
other = {'count':0, 'list':[]}

for qa in qa_dataset:
    query = qa['question']
    query = query.capitalize()
    (start, end) = qa['answer']
    answer_text = qa['context'][start:end]
    context_list = qa['context'].split('. ')
    for sentence in context_list:
        if answer_text in sentence:
            break
    if 'What' in query:
        what['count'] += 1
        what['list'].append(sentence)
    elif 'Why' in query:
        why['count'] += 1
        why['list'].append(sentence)
    elif 'Where' in query:
        where['count'] += 1
        where['list'].append(sentence)
    elif 'When' in query:
        when['count'] += 1
        when['list'].append(sentence)
    elif 'How much' in query:
        how_much['count'] += 1
        how_much['list'].append(sentence)
    elif 'How many' in query:
        how_many['count'] += 1
        how_many['list'].append(sentence)
    elif 'How' in query:
        how['count'] += 1
        how['list'].append(sentence)
    elif 'Which' in query:
        which['count'] += 1
        which['list'].append(sentence)
    elif 'Who' in query:
        who['count'] += 1
        who['list'].append(sentence)
    else:
        other['count'] += 1
        other['list'].append(sentence)

In [7]:
why_wordcount = sort_wordCounter(count_words(why['list']))
what_wordcount = sort_wordCounter(count_words(what['list']))
where_wordcount = sort_wordCounter(count_words(where['list']))
when_wordcount = sort_wordCounter(count_words(when['list']))
howmuch_wordcount = sort_wordCounter(count_words(how_much['list']))
howmany_wordcount = sort_wordCounter(count_words(how_many['list']))
how_wordcount = sort_wordCounter(count_words(how['list']))
which_wordcount = sort_wordCounter(count_words(which['list']))
who_wordcount = sort_wordCounter(count_words(who['list']))
other_wordcount = sort_wordCounter(count_words(other['list']))

wordcount_list = [why_wordcount, what_wordcount, where_wordcount, when_wordcount, howmuch_wordcount, howmany_wordcount, how_wordcount, which_wordcount, who_wordcount, other_wordcount]
print('Num of type of questions: ', len(wordcount_list))

Num of type of questions:  10


In [8]:
def pmi(wordCount,qtype, wordcount_list=wordcount_list):
    pmi_dict = {}
    for word in wordCount:
        # word occurance more than quenstion number, the word must be a very common word,
        # like 'the', 'a', 'it', etc. These words does not help answering qestions

        if wordCount[word]>qtype['count']:
            continue
        # word occurance less than 5% quenstion number
        # it should be a random occurance, we consider there is no relation between such word and the question
        if wordCount[word]<qtype['count']/20:
            break
        else:
            num_word = 0
            vocab_num = 0
            for one_wordcount in wordcount_list:
                if word in one_wordcount:
                    num_word += one_wordcount[word]
                vocab_num += len(one_wordcount)
            p_word = num_word / vocab_num
            conditional_p_word = wordCount[word]/len(wordCount)
            
            pmi_dict[word] = np.log(conditional_p_word/p_word)
    
    return pmi_dict

def get_top50_prompt_words(why_wordcount,qtype):    
    #input: word count dict
    #output: top 50 prompt word list of corresponds to the word count
    why_word_pmiDict = pmi(why_wordcount, qtype) 
    sort_why_pmi = sort_wordCounter(why_word_pmiDict)
    top50_whyPromptWords_list = list(sort_why_pmi)[0:10]
    return top50_whyPromptWords_list

In [9]:
top50_whyPromptWords_list = get_top50_prompt_words(why_wordcount, why)
top50_whatPromptWords_list = get_top50_prompt_words(what_wordcount, what)
top50_wherePromptWords_list = get_top50_prompt_words(where_wordcount, where)
top50_whenPromptWords_list = get_top50_prompt_words(when_wordcount, when)
top50_howmuchPromptWords_list = get_top50_prompt_words(howmuch_wordcount, how_much)
top50_howmanyPromptWords_list = get_top50_prompt_words(howmany_wordcount, how_many)
top50_howPromptWords_list = get_top50_prompt_words(how_wordcount, how)
top50_whichPromptWords_list = get_top50_prompt_words(which_wordcount, which)
top50_whoPromptWords_list = get_top50_prompt_words(who_wordcount, who)
top50_otherPromptWords_list = get_top50_prompt_words(other_wordcount, other)


In [10]:
def generate_Qtype_template(qa_dataset):
    # Random given prompt word for Qtype

    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')
        for sentence in context_list:
            if answer_text in sentence:
                break
    
    for qa in qa_dataset:
        query = qa['question']
        query = query.capitalize()
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')
        for sentence in context_list:
            if answer_text in sentence:
                break

        # question type

        if 'What' in query:
             template_01 = 'To answer a "What" question, you need to look for "{}"'.format(top50_whatPromptWords_list[random.randint(0,len(top50_whatPromptWords_list)-1)]) 
        elif 'Why' in query:
            template_01 = 'To answer a "Why" question, you need to look for "{}"'.format(top50_whyPromptWords_list[random.randint(0,len(top50_whyPromptWords_list)-1)]) 
        elif 'Where' in query:
            template_01 = 'To answer a "Where" question, you need to look for "{}"'.format(top50_wherePromptWords_list[random.randint(0,len(top50_wherePromptWords_list)-1)]) 
        elif 'When' in query:
            template_01 = 'To answer a "When" question, you need to look for "{}"'.format(top50_whenPromptWords_list[random.randint(0,len(top50_whenPromptWords_list)-1)]) 
        elif 'How much' in query:
            template_01 = 'To answer a "How much" question, you need to look for "{}"'.format(top50_howmuchPromptWords_list[random.randint(0,len(top50_howmuchPromptWords_list)-1)]) 
        elif 'How many' in query:
            template_01 = 'To answer a "How many" question, you need to look for "{}"'.format(top50_howmanyPromptWords_list[random.randint(0,len(top50_howmanyPromptWords_list)-1)]) 
        elif 'How' in query:
            template_01 = 'To answer a "How" question, you need to look for "{}"'.format(top50_howPromptWords_list[random.randint(0,len(top50_howPromptWords_list)-1)]) 
        elif 'Which' in query:
            template_01 = 'To answer a "Which" question, you need to look for "{}"'.format(top50_whichPromptWords_list[random.randint(0,len(top50_whichPromptWords_list)-1)]) 
        elif 'Who' in query:
            template_01 = 'To answer a "Who" question, you need to look for "{}"'.format(top50_whoPromptWords_list[random.randint(0,len(top50_whoPromptWords_list)-1)]) 
        else:
            template_01 = 'To answer a "Random" question, you need to look for "{}"'.format(top50_otherPromptWords_list[random.randint(0,len(top50_otherPromptWords_list)-1)]) 


        template = template_01
    
        qa['template'] = template
    
    return qa_dataset

In [15]:
original_squad =  '../../data/NewsQA/testsets/'
Qtype_squad = '../../data/NewsQA/Qtype_testsets/'
filelist = os.listdir(original_squad)
filelist

['newsQA_test_3.json',
 'newsQA_test_2.json',
 'newsQA_test_0.json',
 'newsQA_test_1.json',
 'newsQA_test_4.json']

In [11]:
# f = json.load(open('../../data/NewsQA/processed_newsQA.json'))
# withPrompt = generate_Qtype_template(f)
# json.dump(withPrompt, open('../../data/NewsQA/processed_Qtype_newsQA.json', 'w', encoding='utf-8'))

In [16]:
for file in filelist:
    f = json.load(open(os.path.join(original_squad, file)))
    withPrompt = generate_Qtype_template(f)
    json.dump(withPrompt, open(os.path.join(Qtype_squad, file), 'w', encoding='utf-8'))

In [17]:
dev_qtype = json.load(open(os.path.join(Qtype_squad, 'newsQA_test_3.json'), 'r', encoding='utf-8'))
dev_qtype[1]

{'context': "KATHMANDU, Nepal (CNN) -- Monsoon flooding has left 50,000 people homeless in Nepal and killed at least 74 people in northern India, according to officials.\n\n\n\nMonsoon rain has bought heavy flooding to southern Nepal, pictured, and northern India.\n\n\n\nThe Nepalese army Thursday used helicopters to rescue residents in southeastern Nepal who were stranded on treetops and roofs after monsoon flooding forced more than 50,000 people from their homes.\n\n\n\nIn northern India monsoon flooding destroyed mud huts and killed at least 74 people, officials said. The deaths were in the state of Uttar Pradesh.\n\n\n\nThe latest casualties bring to more than 300 the number of people killed in India since the start of this year's monsoon season.\n\n\n\nNepalese officials said at least four southern villages were inundated with water after the embankment on the Saptakoshi River burst Monday.\n\n\n\nSome 40,000 people were now living in government-run relief camps; others had gone t

### Add phrase template to dataset

In [90]:
def generate_phrase_template_for_domain(qa_dataset):
    pattern = """
        NP: {++}
        NPs: {++}
        Ns: {++}
        """
    pattenParse = nltk.RegexpParser(pattern)

    # count=0
    for qa in qa_dataset:
        query = qa['question']
        (start, end) = qa['answer']
        answer_text = qa['context'][start:end]
        context_list = qa['context'].split('. ')

        tokens = nltk.word_tokenize(query)
        tagged = nltk.pos_tag(tokens)
        cs = pattenParse.parse(tagged)
        phrase_template = '"'
        word_template = '"'
        candidates = []
        for i in cs:
            if type(i) != tuple: 
                for word in i:
                    phrase_template += word[0]+' ' 
                phrase_template += ', '
                # break
            elif i[1] == 'JJ' or i[1] == 'NN' or i[1] == 'NNS':
                candidates.append(i[0])
        phrase_template = phrase_template[:-3]+'"'

        if ', ' in phrase_template:
            template_01 = phrase_template + ' are important phrases. '
        elif ' ' in phrase_template:
            template_01 = phrase_template + ' is an important phrase. '
        elif len(phrase_template)>3:
            template_01 = phrase_template + ' is an important word. '
        else:
            template_01 = 'There is no important phrase in this query. '
            # template_01 = ' '

        if len(candidates)>0:
            for candi in range(len(candidates)):
                word_template+=candidates[candi]+'", '
                # if candi >= 2:
                #     break
            word_template = word_template[:-2]

            if template_01 == 'There is no important phrase in this query. ':
                template_02 = 'But also pay attention to these words: ' + word_template
            else:
                template_02 = 'And also pay attention to these words: ' + word_template

            template = template_01+template_02
        else:
            template = template_01

        qa['template'] = template

    return qa_dataset

In [91]:
Phrase_squad = '../../data/SQuAD_V2/Phrase_SQUAD/'
for file in filelist:
    f = json.load(open(os.path.join(original_squad, file)))
    withPrompt = generate_phrase_template_for_domain(f)
    json.dump(withPrompt, open(os.path.join(Phrase_squad, file), 'w', encoding='utf-8'))

ValueError: Illegal chunk pattern: {++}